# BACKUP BATTERY ESTIMATOR

In [ ]:
%pip install fpdf

import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO
from fpdf import FPDF

st.set_page_config(page_title="Battery Backup Estimator", layout="wide")

st.title("Battery Backup Capacity Estimator")
st.markdown("Estimate required battery capacity for households or industrial use based on realistic parameters.")


User Iputs

In [ ]:
# User Inputs
st.sidebar.header("User Inputs")

application_type = st.sidebar.selectbox("Application Type", ["Household", "Industrial"])
load_entries = st.sidebar.number_input("Number of appliances/machines", min_value=1, max_value=50, value=5)

st.sidebar.markdown("---")
st.sidebar.subheader("Backup & System Parameters")
backup_hours = st.sidebar.number_input("Desired Backup Duration (hrs)", 1, 48, value=6)
autonomy_days = st.sidebar.number_input("Days of Autonomy (off-grid)", 1, 7, value=1)
battery_voltage = st.sidebar.selectbox("Battery Bank Voltage (V)", [12, 24, 48], index=2)
battery_efficiency = st.sidebar.slider("Battery Efficiency (%)", 70, 100, value=90)

Appliances Inputs

In [ ]:
# Appliance Inputs
st.subheader("🔌 Load/Appliance Details")

appliance_data = []
for i in range(load_entries):
    col1, col2, col3 = st.columns(3)
    name = col1.text_input(f"Appliance {i+1} Name", value=f"Load {i+1}")
    power = col2.number_input(f"Power (W) for {name}", min_value=1.0, value=100.0, key=f"power{i}")
    usage = col3.number_input(f"Hours/Day for {name}", min_value=0.1, value=4.0, key=f"usage{i}")
    appliance_data.append({"Appliance": name, "Power_W": power, "Hours_Per_Day": usage})

df = pd.DataFrame(appliance_data)
df["Energy_Wh_Per_Day"] = df["Power_W"] * df["Hours_Per_Day"]
total_energy_wh = df["Energy_Wh_Per_Day"].sum()
required_capacity_Wh = (total_energy_wh * autonomy_days) / (battery_efficiency / 100)
required_capacity_Ah = required_capacity_Wh / battery_voltage

st.write("### 🔍 Energy Consumption Summary")
st.dataframe(df, use_container_width=True)
st.success(f"Total Daily Load: {total_energy_wh:.2f} Wh")
st.success(f"Required Battery Capacity: {required_capacity_Wh:.2f} Wh / {required_capacity_Ah:.2f} Ah @ {battery_voltage}V")


Ploting the Load Distribution Chart

In [ ]:
# Plot
st.write("### Load Distribution Chart")
fig, ax = plt.subplots(figsize=(8, 4))
df.plot(kind='bar', x='Appliance', y='Energy_Wh_Per_Day', ax=ax, legend=False, color="orange")
ax.set_ylabel("Energy (Wh)")
st.pyplot(fig)

Excel Export

In [ ]:
%pip install xlsxwriter

# Excel Export
def to_excel(dataframe):
    output = BytesIO()
    with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
        dataframe.to_excel(writer, index=False, sheet_name="Load Data")
        workbook = writer.book
        worksheet = writer.sheets["Load Data"]
        worksheet.write("G2", "Total Daily Load (Wh)")
        worksheet.write("G3", total_energy_wh)
        worksheet.write("G4", "Battery Capacity Needed (Ah)")
        worksheet.write("G5", required_capacity_Ah)
        writer.save()
    return output.getvalue()

excel_data = to_excel(df)
st.download_button("Download Excel Report", data=excel_data, file_name="battery_capacity_report.xlsx")

PDF Export

In [ ]:
# PDF Export
def generate_pdf():
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt="Battery Capacity Estimation Report", ln=True, align="C")
    pdf.ln(10)
    for index, row in df.iterrows():
        pdf.cell(200, 10, txt=f"{row['Appliance']} - {row['Power_W']}W x {row['Hours_Per_Day']}hrs = {row['Energy_Wh_Per_Day']:.2f}Wh", ln=True)
    pdf.ln(10)
    pdf.cell(200, 10, txt=f"Total Daily Energy Consumption: {total_energy_wh:.2f} Wh", ln=True)
    pdf.cell(200, 10, txt=f"Required Battery Capacity: {required_capacity_Ah:.2f} Ah @ {battery_voltage}V", ln=True)
    pdf.ln(10)
    pdf_bytes = pdf.output(dest='S').encode('latin-1')
    return pdf_bytes

pdf_data = generate_pdf()
st.download_button("Download PDF Report", data=pdf_data, file_name="battery_report.pdf", mime="application/pdf")

st.markdown("---")
st.caption("Developed by Victor | Streamlit + Python | v1.0")
# This code is a Streamlit application that estimates battery backup capacity based on user inputs for appliances and system parameters.
# It allows users to input appliance details, calculates total energy consumption, and provides options to download